In [ ]:
#not sure if we still need this

def build_worker_agent():
    """Builds the LangGraph that functions as the ReAct Worker Agent."""
    
    # 1. The toolkit for the agent
    tools = [get_knowledge_base_chunks, geolocate_ip, get_places]
    
    # 2. The agent's "brain" (LLM)
    # Note: "gpt-4o-mini" is the current standard name for this model class.
    # Using the name from your file for consistency.
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0, api_key=os.getenv("OPENAI_API_KEY"))
    
    # Bind the tools to the LLM, making it "tool-aware"
    llm_with_tools = llm.bind_tools(tools)
    
    # 3. The agent's "instructions" (Prompt)
    prompt = ChatPromptTemplate.from_messages([
       ("system", """
        You are a specialized research agent. Your job is to receive a task and use your tools to find the required information.
        After using your tools, you must compile all the results into a single, structured JSON object.
        Do not respond in conversational language; your output must be only the final JSON report.
        """),
       ("human", "{input}"), 
       ("placeholder", "{agent_scratchpad}"),
    ])
    
    # The runnable chain for the agent's brain
    chain = prompt | llm_with_tools

In [ ]:

from typing import TypedDict

class WorkerAgentState(TypedDict): 
    from typing import TypedDict
    messages: list[str] # We need this messages key for our react agent to use
    workeragentsummary: str
    



In [ ]:
worker_agent_prompt = """
    You are a helpful assistant who is a research agent. You have access to the static knowledge base that has information regarding the regulations and laws of recycling and waste disposal in three cities: NYC, LA, and Chicago. You also have access to the google_search tool the get_knowledge_basez_chunks tool. You will provide StructuredOutput from the knowledge base that you have been provided and relay it to the Supervisor Agent.
"""

In [ ]:
%pip install langchain

from langchain.chat_models import init_chat_model

llm = init_chat_model(model_provider="openai", model="gpt-4.1-mini")

#define the tools as lists
worker_agent_tools = [get_knowledge_base_chunks, geolocate_ip, get_places, google_search]
worker_agent = create_react_agent(model=llm, tools=worker_agent_tools, prompt=worker_agent_prompt)

def worker_agent_node(state: WorkerAgentState, config: Optional[RunnableConfig]): 
    response = worker_agent.invoke({"messages": state["messages"][-1]}) #Passes in the latest message from state
    result = response["messages"][-1] 
    return {
        "messages": state["messages"] + [result], 
        "worker_agent_output": result
    }
